In [226]:
import requests
import json
import pandas as pd
import numpy as np
from datetime import date, timedelta, timezone, datetime
from sklearn import preprocessing

In [227]:
url = 'https://statsapi.web.nhl.com/api/v1/'
schedule = 'schedule'
date_query = 'date='
linescore_query = 'expand=schedule.linescore'
season_query = '?season=20172018'
today_date = str(date.today() - timedelta(days=1,hours=5))
print(today_date)

2022-04-25


In [228]:
#get json response as text
#compose url for yesterday's schedule with linescore
url_linescore = url+schedule+'?'+linescore_query+'&'+date_query+today_date
print(url_linescore)
response = requests.get(url_linescore).text

https://statsapi.web.nhl.com/api/v1/schedule?expand=schedule.linescore&date=2022-04-25


In [229]:
dict_response = json.loads(response)

In [230]:
dict_response

{'copyright': 'NHL and the NHL Shield are registered trademarks of the National Hockey League. NHL and NHL team marks are the property of the NHL and its teams. © NHL 2022. All Rights Reserved.',
 'totalItems': 1,
 'totalEvents': 0,
 'totalGames': 1,
 'totalMatches': 0,
 'metaData': {'timeStamp': '20220426_092741'},
 'wait': 10,
 'dates': [{'date': '2022-04-25',
   'totalItems': 1,
   'totalEvents': 0,
   'totalGames': 1,
   'totalMatches': 0,
   'games': [{'gamePk': 2021021271,
     'link': '/api/v1/game/2021021271/feed/live',
     'gameType': 'R',
     'season': '20212022',
     'gameDate': '2022-04-26T00:00:00Z',
     'status': {'abstractGameState': 'Final',
      'codedGameState': '7',
      'detailedState': 'Final',
      'statusCode': '7',
      'startTimeTBD': False},
     'teams': {'away': {'leagueRecord': {'wins': 25,
        'losses': 44,
        'ot': 11,
        'type': 'league'},
       'score': 1,
       'team': {'id': 4,
        'name': 'Philadelphia Flyers',
        'li

In [231]:
total_games = dict_response['dates'][0]['totalGames']

In [232]:
total_games

1

In [233]:
all_games = dict_response['dates'][0]['games']

In [234]:
home_team_scores = []
away_team_scores = []
home_teams = []
away_teams = []
goals_total = []
shots_home = []
shots_away = []
shots_total = []
game_status = []
gameId = []

for i in range(0,total_games):
    home_team_scores.append((all_games[i]['teams']['home']['score']))
    home_teams.append((all_games[i]['teams']['home']['team']['name']))
    away_team_scores.append((all_games[i]['teams']['away']['score']))
    away_teams.append((all_games[i]['teams']['away']['team']['name']))
    goals_total.append((all_games[i]['teams']['home']['score'])+(all_games[i]['teams']['away']['score']))
    shots_home.append(all_games[i]['linescore']['teams']['home']['shotsOnGoal'])
    shots_away.append(all_games[i]['linescore']['teams']['away']['shotsOnGoal'])
    shots_total.append(all_games[i]['linescore']['teams']['home']['shotsOnGoal']+all_games[i]['linescore']['teams']['away']['shotsOnGoal'])
    game_status.append(all_games[i]['linescore']['currentPeriodOrdinal'])
    gameId.append(str(all_games[i]['gamePk']))


In [235]:
#normalize goals
goals_total_np = np.array(goals_total).reshape(-1,1)

In [236]:
scaler = preprocessing.MinMaxScaler()
goals_normalized = scaler.fit_transform(goals_total_np)
goals_normalized = np.round(goals_normalized,4)

In [237]:
#prepare a list of lists for dataframe conversion
data = []
data.append(home_teams)
data.append(away_teams)
data.append(goals_total)
data.append(goals_normalized)
data.append(shots_total)
#data.append(game_status)
data.append(gameId)

In [238]:
team_scores_df = pd.DataFrame(data).transpose()

In [239]:
team_scores_df.columns=['home team','away team', 'goal total','normalized goals','shots total', 'gameId']

In [240]:
team_scores_df.set_index('gameId')

,home team,away team,goal total,normalized goals,shots total
gameId,,,,,
2021021271,Chicago Blackhawks,Philadelphia Flyers,4,[0.0],67
